## 1. Data preparation


In [155]:
import torch
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [156]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### Q1_a: Print the size of the vocabulary of the above tokenizer.

In [157]:
print('The size of the vocabulary of the tokenizer is: {}'.format(tokenizer.vocab_size))

The size of the vocabulary of the tokenizer is: 30522


In [129]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


In [130]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

[CLS] [SEP] [PAD] [UNK]
101 102 0 100
512


In [131]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [132]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

### Q1_b. Print the number of data points in the train, test, and validation sets.

In [133]:
print('The number of data points in the train sets is: {}'.format(len(train_data)))
print('The number of data points in the test sets is: {}'.format(len(test_data)))
print('The number of data points in the validation sets is: {}'.format(len(valid_data)))

The number of data points in the train sets is: 17500
The number of data points in the test sets is: 25000
The number of data points in the validation sets is: 7500


In [134]:
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)


defaultdict(None, {'neg': 0, 'pos': 1})


In [160]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'mps')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## 2. Model preparation


In [136]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [137]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,bert,hidden_dim,output_dim,n_layers,bidirectional,dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

### Q2a: Instantiate the above model by setting the right hyperparameters.
* the BERT embedding (whose weights are frozen)
* a bidirectional GRU with 2 layers, with hidden dim 256 and dropout=0.25.
* a linear layer on top which does binary sentiment classification.


In [138]:
# insert code here
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

### Q2b: Print the number of trainable parameters in this model.


In [139]:
# insert code here.

num_params = sum(i.numel() for i in model.parameters() if i.requires_grad)
print('The number of trainable parameters in this model is: {}'.format(num_params))

The number of trainable parameters in this model is: 112241409


In [140]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

### Q2c: After freezing the BERT weights/biases, print the number of remaining trainable parameters.

In [141]:
num_params = sum(i.numel() for i in model.parameters() if i.requires_grad)
print('The number of trainable parameters in this model is: {}'.format(num_params))

The number of trainable parameters in this model is: 2759169


## 3. Train the Model

We will use:
* the Binary Cross Entropy loss function: `nn.BCEWithLogitsLoss()`
* the Adam optimizer

and run it for 2 epochs (that should be enough to start getting meaningful results).

In [142]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [143]:
criterion = nn.BCEWithLogitsLoss()

In [158]:
model = model.to(device)
criterion = criterion.to(device)

### Q3. 
* calculating accuracy. 
* training for a single epoch, and reporting loss/accuracy.
* performing an evaluation epoch, and reporting loss/accuracy.
* calculating running times.

In [145]:
def binary_accuracy(preds, y):
    
    # Q3a. Compute accuracy (as a number between 0 and 1)

    # ...    
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    
    return acc


In [151]:
def train(model, iterator, optimizer, criterion):
    
    # Q3b. Set up the training function

    # ...
    epoch_loss = 0
    epoch_acc =0
    
    model.train()
    
    for batch in iterator:
        
        text, label = batch.text, batch.label # Move data to device
        
        optimizer.zero_grad() # zero out any gradient values from the previous iteration
        
        pred = model(text).squeeze(1) # forward propgation
        
        loss = criterion(pred,label) # calculate loss
        loss.backward() # back propagation
        
        optimizer.step() # update the weights of our trainable parameters
        
        acc = binary_accuracy(pred,label)

        epoch_loss += loss.item()
        epoch_acc +=acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [152]:
def evaluate(model, iterator, criterion):
    
    # Q3c. Set up the evaluation function. 

    # ...
    epoch_loss = 0
    epoch_acc =0
    
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, label = batch.text.to(device),batch.label.to(device) # Move data to device
            
            pred = model(text).squeeze(1) # forward propagation
            
            loss = criterion(pred,label) # calculate loss

            acc = binary_accuracy(pred,label) #calculate accuracy

            epoch_loss += loss.item()
            epoch_acc +=acc.item()
    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [153]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [154]:
N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    # Q3d. Perform training/valudation by using the functions you defined earlier.

    start_time = time.time() 
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, test_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

KeyboardInterrupt: 

In [163]:
model.load_state_dict(torch.load('model.pt'))



RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.